In [1]:
from gpt4all import GPT4All

In [2]:
model_name = "Phi-3-mini-4k-instruct.Q4_0.gguf"

model = GPT4All(model_name)

In [3]:
with model.chat_session():
    print(model.generate("What is 500 * 500"))

 The product of 500 multiplied by 500 is:

500 * 500 = 250,000.


In [4]:
model.retrieve_model(model_name)

{'order': 'q',
 'md5sum': 'f8347badde9bfc2efbe89124d78ddaf5',
 'name': 'Phi-3 Mini Instruct',
 'filename': 'Phi-3-mini-4k-instruct.Q4_0.gguf',
 'filesize': '2176181568',
 'requires': '2.7.1',
 'ramrequired': '4',
 'parameters': '4 billion',
 'quant': 'q4_0',
 'type': 'Phi-3',
 'description': '<ul><li>Very fast responses</li><li>Chat based model</li><li>Accepts system prompts in Phi-3 format</li><li>Trained by Microsoft</li><li>License: <a href="https://opensource.org/license/mit">MIT</a></li><li>No restrictions on commercial use</li></ul>',
 'url': 'https://gpt4all.io/models/gguf/Phi-3-mini-4k-instruct.Q4_0.gguf',
 'promptTemplate': '<|user|>\n{0}<|end|>\n<|assistant|>\n{1}<|end|>\n',
 'systemPrompt': '',
 'chatTemplate': "{{- bos_token }}\n{%- for message in messages %}\n    {{- '<|' + message['role'] + '|>\\n' + message['content'] + '<|end|>\\n' }}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|assistant|>\\n' }}\n{%- else %}\n    {{- eos_token }}\n{%- endif %}",
 'path'

In [5]:
with model.chat_session():
    print(model.generate('Hello, My Name is Osher'))
    print(model.generate('What is my name?'))

 Hi there! It's great to meet you. Although I don't have a name myself as an AI, feel free to call me Assistant. How can I help you today? If you prefer using the nickname "Osher," that works too for our conversation here and now! Just let me know how things are going or what information you need assistance with.
 Hello again! As an AI, I don't have personal knowledge of individuals unless shared during a specific interaction. If we had been conversing earlier about your preferred nickname being "Osher," then that would be the friendly identifier for our conversation. However, without prior context or information from you directly in this session, my name remains simply as an AI Assistant designed to help answer questions and provide guidance on a wide range of topics!


In [6]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


pdf_loader = PyPDFLoader(fr'c:\\Users\\Osher N Boudara\\OneDrive\\Desktop_Ubuntu\\Projects\\Development\\resume_app\\resume\\osher_boudara_resume.pdf')

In [7]:
resume_doc = pdf_loader.load()




In [8]:
resume_doc

[Document(metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Osher_Boudara_Resume', 'source': 'c:\\\\Users\\\\Osher N Boudara\\\\OneDrive\\\\Desktop_Ubuntu\\\\Projects\\\\Development\\\\resume_app\\\\resume\\\\osher_boudara_resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content="OSHER  BOUDARA  osherboudara99@gmail.com |  818.809.4261    \nEXPERIENCE  Cognizant   -   Data  Scientist  [ [ B R ] ]                                Aug.  2022  -  Present ●  Leading  and  developing  projects  in  multiple  data  science  and  engineering  teams  in  the  crop  science  division  of  a  global  \nFortune\n \n500\n \nclient\n \ncompany,\n \nadapting\n \nto\n \nevolving\n \nrequirements\n \nand\n \ndelivering\n \nsolutions\n \nto\n \nstakeholders.\n ●  Implementing  data  science  life  cycle  components:  automated  ETL  pipelines,  MLOps,  cloud-computing  code  development,  \ncloud\n \ntechnologies,\n \nversion\n

In [16]:
text_split = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 100)

In [17]:
docs = text_split.split_documents(resume_doc)

In [18]:
docs

[Document(metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Osher_Boudara_Resume', 'source': 'c:\\\\Users\\\\Osher N Boudara\\\\OneDrive\\\\Desktop_Ubuntu\\\\Projects\\\\Development\\\\resume_app\\\\resume\\\\osher_boudara_resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='OSHER  BOUDARA  osherboudara99@gmail.com |  818.809.4261'),
 Document(metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Osher_Boudara_Resume', 'source': 'c:\\\\Users\\\\Osher N Boudara\\\\OneDrive\\\\Desktop_Ubuntu\\\\Projects\\\\Development\\\\resume_app\\\\resume\\\\osher_boudara_resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='EXPERIENCE  Cognizant   -   Data  Scientist  [ [ B R ] ]                                Aug.  2022  -  Present ●  Leading  and  developing  projects  in  multiple  data  science  and  engineering'),
 Document(metadata={'pr

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Convert resume text into embeddings
vector_db = FAISS.from_documents(docs, embedding_model)

# # Save the vector database for future use
# vector_db.save_local("faiss_index_resume")

In [ ]:
def get_relevant_context(query):
    """Retrieve relevant sections from the resume."""
    search_results = vector_db.similarity_search(query, k=2)  # Get top 2 matches
    context = "\n".join([doc.page_content for doc in search_results])
    return context

def chat_with_resume(query):
    """Retrieve relevant context and generate a response."""
    context = get_relevant_context(query)
    prompt = f"Using the following resume details, answer the question:\n\n{context}\n\nQuestion: {query}"
    
    response = model.generate(prompt, max_tokens=200)
    return response

# Example Chatbot Query
user_query = "What programming languages does Osher know?"
response = chat_with_resume(user_query)
print("Chatbot:", response)

Chatbot:  List them in alphabetical order, separated by commas.
<|assistant|> Based on the provided resume details and assuming that GitHub repositories represent his coding skills, it appears that Osher knows the following programming languages (alphabetically): C++, Java, Python. However, please note this is an assumption as explicit language proficiency was not mentioned in the given information.


: 